In [4]:
!pip install transformers 

     |████████████████████████████████| 3.8 MB 5.0 MB/s 
     |████████████████████████████████| 895 kB 37.8 MB/s 
     |████████████████████████████████| 6.5 MB 25.9 MB/s 
     |████████████████████████████████| 596 kB 48.2 MB/s 
     |████████████████████████████████| 67 kB 3.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
 import torch
torch.cuda.is_available()

False

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [5]:
!pip install datasets

     |████████████████████████████████| 325 kB 5.2 MB/s 
     |████████████████████████████████| 136 kB 45.9 MB/s 
     |████████████████████████████████| 1.1 MB 25.1 MB/s 
     |████████████████████████████████| 212 kB 48.4 MB/s 
     |████████████████████████████████| 127 kB 50.3 MB/s 
     |████████████████████████████████| 271 kB 49.4 MB/s 
     |████████████████████████████████| 94 kB 796 kB/s 
     |████████████████████████████████| 144 kB 41.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install accelerate

     |████████████████████████████████| 65 kB 2.6 MB/s 


In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()

device = accelerator.device

In [ ]:
device

device(type='cuda')

In [6]:
import logging
import math
import os

import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss

from transformers.activations import gelu_new
from transformers import GPT2Model, GPT2Config
import argparse
import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple
import json
#import wget
import math

import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from torch.distributed import get_rank, get_world_size
from tqdm import tqdm, trange

from torch.nn import functional as F
from torch.nn.modules.normalization import LayerNorm
from torch.nn.modules import ModuleList
import copy

from transformers import AdamW, GPT2Config, GPT2LMHeadModel, GPT2Tokenizer

In [7]:
class Conv1D(nn.Module):
    def __init__(self, nx, nf):
        super().__init__()
        self.nf = nf
        w = torch.empty(nx, nf)
        nn.init.normal_(w, std=0.02)
        self.weight = nn.Parameter(w)
        self.bias = nn.Parameter(torch.zeros(nf))

    def forward(self, x):
        size_out = x.size()[:-1] + (self.nf,)
        x = torch.addmm(self.bias, x.view(-1, x.size(-1)), self.weight)
        x = x.view(*size_out)
        return x
class FeedForward(nn.Module):
    def __init__(self, dropout, d_model=768, nx=768*4):
        super().__init__()
        self.c_fc    = Conv1D(d_model, nx)
        self.c_proj  = Conv1D(nx, d_model)
        self.act     = F.gelu
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        return self.dropout(self.c_proj(self.act(self.c_fc(x))))
    


def _get_clones(module, n):
    return ModuleList([copy.deepcopy(module) for i in range(n)])
    
class Attention(nn.Module):
    def __init__(self, d_model=768, n_head=12, n_ctx=1024, d_head=64, bias=True, scale=False):
        super().__init__()
        self.n_head  = n_head
        self.d_model = d_model
        self.c_attn  = Conv1D(d_model, d_model*3)
        self.scale   = scale
        self.softmax = nn.Softmax(dim=-1)
        self.register_buffer("bias", torch.tril(torch.ones(n_ctx, n_ctx)).view(1, 1, n_ctx, n_ctx))
        self.dropout = nn.Dropout(0.1)
        self.c_proj  = Conv1D(d_model, d_model)
        
    def split_heads(self, x):
        "return shape [`batch`, `head`, `sequence`, `features`]"
        new_shape = x.size()[:-1] + (self.n_head, x.size(-1)//self.n_head) 
        x = x.view(*new_shape)
        return x.permute(0, 2, 1, 3) 
    
    def _attn(self, q, k, v, attn_mask=None):
        scores  = torch.matmul(q, k.transpose(-2, -1))
        if self.scale: scores = scores/math.sqrt(v.size(-1))
        nd, ns  = scores.size(-2), scores.size(-1)
        if attn_mask is not None: scores = scores + attn_mask
        scores  = self.softmax(scores)
        scores  = self.dropout(scores)
        outputs = torch.matmul(scores, v)
        return outputs
    
    def merge_heads(self, x):
        x         = x.permute(0, 2, 1, 3).contiguous()
        new_shape = x.size()[:-2] + (x.size(-2)*x.size(-1),)
        return x.view(*new_shape)
        
    def forward(self, x):
        x        = self.c_attn(x) #new `x` shape - `[1,3,2304]`
        q, k, v  = x.split(self.d_model, dim=2)
        q, k, v  = self.split_heads(q), self.split_heads(k), self.split_heads(v)
        out      = self._attn(q, k, v)
        out      = self.merge_heads(out)
        out      = self.c_proj(out)
        return out
    
class TransformerBlock(nn.Module):
    def __init__(self, d_model=768, n_head=12, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.attn        = Attention(d_model=768, n_head=12, d_head=64, n_ctx=1024, bias=True, scale=False)
        self.feedforward = FeedForward(dropout=0.1, d_model=768, nx=768*4)
        self.ln_1        = LayerNorm(d_model)
        self.ln_2        = LayerNorm(d_model)
                
    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.feedforward(self.ln_2(x))
        return x
    
class MyModel(nn.Module):
    def __init__(self, nlayers=12, n_ctx=1024, d_model=768, vcb_sz=50257):
        super(MyModel, self).__init__()
        self.nlayers = nlayers
        block        = TransformerBlock(d_model=768, n_head=12, dropout=0.1)
        self.h       = _get_clones(block, nlayers)
        self.wte     = nn.Embedding(vcb_sz, d_model)
        self.wpe     = nn.Embedding(n_ctx, d_model)
        self.drop    = nn.Dropout(0.1)
        self.ln_f    = LayerNorm(d_model)
        self.out     = nn.Linear(d_model, vcb_sz, bias=False)
        self.loss_fn = nn.CrossEntropyLoss()
        self.init_weights()
        #self.temp = TransformerBlockc()
        from transformers import GPT2Tokenizer
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        #self.n  = torch.tensor(tokenizer.encode('Negative')).type(torch.LongTensor)
    
    def init_weights(self):
        self.out.weight = self.wte.weight
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding, Conv1D)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, (nn.Linear, Conv1D)) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
            
    def prepare_embeds_inputs_for_generation(self, inputs_embeds, **kwargs):
        # only last token for inputs_ids if past is defined in kwargs
        if "past" in kwargs and kwargs["past"]:
            inputs_embeds = inputs_embeds[:, -1:, :]

        inputs = {"inputs_embeds": inputs_embeds}
        inputs.update(kwargs)
        return inputs

    def prepare_hidden_state_inputs_for_generation(self, input_hidden_state, **kwargs):
        # only last token for inputs_ids if past is defined in kwargs
        if "past" in kwargs and kwargs["past"]:
            input_hidden_state = input_hidden_state[:, -1:, :]

        inputs = {"input_hidden_state": input_hidden_state}
        inputs.update(kwargs)
        return inputs
            

    def forward_half1(self,src, labels=None, pos_ids=None, inputs_embeds=None):
        if(inputs_embeds): #x=src is input embeds
            if pos_ids is None: pos_ids = torch.arange(0, src.size(-2)).unsqueeze(0)
            wpe2 = nn.Embedding(src.size(-2), 768).to(device)
            pos_ids = pos_ids.to(device)
            position_embeds = wpe2(pos_ids).to(device)
            inp = self.drop(src + position_embeds)
            
        else:
            if pos_ids is None: pos_ids = torch.arange(0, src.size(-1)).unsqueeze(0)
            pos_ids = pos_ids.to(device)
            position_embeds = self.wpe(pos_ids)
            position_embeds=position_embeds.to(device)

            inp = self.drop((self.wte(src)+position_embeds))
            
        #inp = self.drop((self.wte(src)+self.wpe(pos_ids)))
        for i in range(6): inp = self.h[i](inp)
        return inp
        
    
    def forward_half2(self, inp,labels=None, pos_ids=None,lm_logit_first_index=0,lm_logit_last_index=-1,
                     lm_labels_first_index=1, lm_labels_last_index=None):
        for i in range(6,12): inp = self.h[i](inp)
        inp     = self.ln_f(inp)
        logits  = self.out(inp)
        outputs = (logits,) + (inp,)
        
        if labels is not None:
            # Shift so that tokens < n predict n
            shift_logits = logits[..., lm_logit_first_index:lm_logit_last_index, :].contiguous() # default lm_logit_first_index=0, lm_logit_last_index=-1,
            shift_labels = labels[..., lm_labels_first_index:lm_labels_last_index].contiguous() # default lm_labels_first_index=1, lm_labels_last_index=None,

            loss_fct = CrossEntropyLoss()

            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            outputs = (loss,) + outputs

        return outputs,logits
    """
   
    def cocon(self,inp,content):
        #content = torch.tensor(self.tokenizer.encode(content))#.long()
        content = self.forward_half1(content)
        x  = self.temp(inp,content)
        h_t_2 = x[:,:-1,:]
        h_t_1 = x[:,-1,:]
        h_t_1 = torch.unsqueeze(h_t_1, 1)
        h_ = torch.cat([h_t_2,h_t_1],dim=1)
        return h_
        """ 
    
    def forward(self, x,labels=None,path='all',lm_logit_first_index=0,lm_logit_last_index=-1,
                     lm_labels_first_index=1, lm_labels_last_index=None, inputs_embeds=None):
        
        if path=='all':
            x = self.forward_half1(x,inputs_embeds=inputs_embeds)
            #x = self.cocon(x,content)
            x = self.forward_half2(x,labels)
        elif path=='half1':
            x = self.forward_half1(x,inputs_embeds=inputs_embeds)
        elif path=='half2':
            x = self.forward_half2(x,labels,lm_logit_first_index=0,lm_logit_last_index=-1,
                     lm_labels_first_index=1, lm_labels_last_index=None)
        #elif path=='cocon':
        #    x = self.cocon(x,content)
        else:
            raise NotImplementedError
        return x

In [8]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [9]:
model = MyModel()
model_dict = model.state_dict()

In [10]:
for param in model.parameters():
    #print(param)
    param.requires_grad = False

In [12]:
from google.colab import drive

mount='/content/gdrive'
print("Colab: mounting Google drive on ", mount)

drive.mount(mount)
# Switch to the directory on the Google Drive that you want to use
import os
drive_root = mount + "/My Drive/NLP-project-2022/"
%cd $drive_root

Colab: mounting Google drive on  /content/gdrive
Mounted at /content/gdrive
/content/gdrive/My Drive/NLP-project-2022


In [13]:
%pwd

'/content/gdrive/My Drive/NLP-project-2022'

In [14]:
state_dict = torch.load("gpt_wt/gpt2-pytorch_model.bin") #pretrained weights

old_keys = []
new_keys = []
for key in state_dict.keys(): 
    if "mlp" in key: #The hugging face state dict references the feedforward network as mlp, need to replace to `feedforward` be able to reuse these weights
        new_key = key.replace("mlp", "feedforward")
        new_keys.append(new_key)
        old_keys.append(key)

for old_key, new_key in zip(old_keys, new_keys): 
    state_dict[new_key]=state_dict.pop(old_key)

pretrained_dict = {k: v for k, v in state_dict.items() if k in model_dict}

In [15]:
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)

<All keys matched successfully>

In [16]:
model = model.to(device)

In [17]:
model.eval()
model.zero_grad()

In [18]:
logger = logging.getLogger(__name__)

In [19]:
from transformers import get_linear_schedule_with_warmup
from transformers.modeling_utils import Conv1D, PreTrainedModel, SequenceSummary, prune_conv1d_layer



In [20]:
class MLP(nn.Module):
    def __init__(self, n_state, config):  # in MLP: n_state=3072 (4 * n_embd)
        super().__init__()
        nx = config.n_embd
        self.c_fc = Conv1D(n_state, nx)
        self.c_proj = Conv1D(nx, n_state)
        self.act = gelu_new
        self.dropout = nn.Dropout(config.resid_pdrop)

    def forward(self, x):
        h = self.act(self.c_fc(x))
        h2 = self.c_proj(h)
        return self.dropout(h2)

In [21]:
class CoconAttention(nn.Module):
    def __init__(self, nx, n_ctx, config, scale=False):
        super().__init__()
        self.output_attentions = config.output_attentions

        n_state = nx  # in Attention: n_state=768 (nx=n_embd)
        # [switch nx => n_state from Block to Attention to keep identical to TF implem]
        assert n_state % config.n_head == 0
        self.register_buffer("bias", torch.tril(torch.ones(n_ctx, n_ctx)).view(1, 1, n_ctx, n_ctx))

        self_token_mask = torch.ones(n_ctx, n_ctx)
        self_token_mask.fill_diagonal_(0)
        self.register_buffer("self_token_mask", self_token_mask.view(1, 1, n_ctx, n_ctx))
        self.n_head = config.n_head
        self.split_size = n_state
        self.scale = scale

        self.ref_source_attn = Conv1D(n_state * 2, nx)
        self.c_attn = Conv1D(n_state * 3, nx) # input has dim of nx
        self.c_proj = Conv1D(n_state, nx)
        self.attn_dropout = nn.Dropout(config.attn_pdrop)
        self.resid_dropout = nn.Dropout(config.resid_pdrop)
        self.pruned_heads = set()

    def prune_heads(self, heads):
        if len(heads) == 0:
            return
        mask = torch.ones(self.n_head, self.split_size // self.n_head)
        heads = set(heads) - self.pruned_heads  # Convert to set and emove already pruned heads
        for head in heads:
            # Compute how many pruned heads are before the head and move the index accordingly
            head = head - sum(1 if h < head else 0 for h in self.pruned_heads)
            mask[head] = 0
        mask = mask.view(-1).contiguous().eq(1)
        index = torch.arange(len(mask))[mask].long()
        index_attn = torch.cat([index, index + self.split_size, index + (2 * self.split_size)])

        # Prune conv1d layers
        self.c_attn = prune_conv1d_layer(self.c_attn, index_attn, dim=1)
        self.c_proj = prune_conv1d_layer(self.c_proj, index, dim=0)

        # Update hyper params
        self.split_size = (self.split_size // self.n_head) * (self.n_head - len(heads))
        self.n_head = self.n_head - len(heads)
        self.pruned_heads = self.pruned_heads.union(heads)

    def _attn(self, q, k, v, attention_mask=None, head_mask=None, cs_self_attn_mask_prob=0, history_seq_len=None, context_seq_present=True, context_seq_len=0, context_attn_bias=0, context_seq_len_list=None):
        w = torch.matmul(q, k)
        if self.scale:
            w = w / math.sqrt(v.size(-1))
        nd, ns = w.size(-2), w.size(-1)
        b = self.bias[:, :, ns - nd : ns, :ns]
        w = w * b - 1e4 * (1 - b)

        # self_token_mask computation
        if cs_self_attn_mask_prob > 0 and context_seq_present:
            if history_seq_len == 0:
                history_seq_offset = 0
            else:
                history_seq_offset = history_seq_len - 1
            self_token_mask = self.self_token_mask[:, :, :nd, history_seq_offset:history_seq_offset+ns]
            self_token_mask = self_token_mask.repeat(w.shape[0],1,1,1)

            if cs_self_attn_mask_prob != 1:
                # compute unmasked indices
                self_token_unmask_prob = 1 - cs_self_attn_mask_prob
                unmask_prob_matrix = torch.full(self_token_mask.shape[:-1], self_token_unmask_prob)
                unmasked_indices = torch.bernoulli(unmask_prob_matrix).bool()
                self_token_mask[unmasked_indices] = 1

            w = w * self_token_mask - 1e4 * (1 - self_token_mask)
            
        
        if context_attn_bias != 0:
            if context_seq_len_list is None:
                context_attn_bias_mask = torch.ones(w.shape) # N, H, Q, V
                context_attn_bias_mask[:,:,:, :context_seq_len] = 0
                context_attn_bias_mask = context_attn_bias_mask.to(w.device)
                w = w + context_attn_bias * (1 - context_attn_bias_mask)     
            else:
                current_context_start_ind = 0
                for cs_ind, current_context_seq_len in enumerate(context_seq_len_list):
                    current_context_attn_bias = context_attn_bias[cs_ind]
                    context_attn_bias_mask = torch.ones(w.shape)
                    context_attn_bias_mask[:,:,:, current_context_start_ind:(current_context_start_ind+current_context_seq_len)] = 0
                    context_attn_bias_mask = context_attn_bias_mask.to(w.device)
                    w = w + current_context_attn_bias * (1 - context_attn_bias_mask)
                    current_context_start_ind = current_context_start_ind + current_context_seq_len

            
        if attention_mask is not None:
            # Apply the attention mask
            w = w + attention_mask

        w = nn.Softmax(dim=-1)(w)
        w = self.attn_dropout(w)

        # Mask heads if we want to
        if head_mask is not None:
            w = w * head_mask

        outputs = [torch.matmul(w, v)]
        if self.output_attentions:
            outputs.append(w)
        return outputs

    def merge_heads(self, x):
        x = x.permute(0, 2, 1, 3).contiguous()
        new_x_shape = x.size()[:-2] + (x.size(-2) * x.size(-1),)
        return x.view(*new_x_shape)  # in Tensorflow implem: fct merge_states

    def split_heads(self, x, k=False):
        new_x_shape = x.size()[:-1] + (self.n_head, x.size(-1) // self.n_head)
        x = x.view(*new_x_shape)  # in Tensorflow implem: fct split_states
        if k:
            return x.permute(0, 2, 3, 1)  # (batch, head, head_features, seq_length)
        else:
            return x.permute(0, 2, 1, 3)  # (batch, head, seq_length, head_features)

    def forward(self, x, context_seq, layer_past=None, attention_mask=None, head_mask=None, cs_self_attn_mask_prob=0, history_seq_len=None, context_attn_bias=0, context_seq_len_list=None):
        x = self.c_attn(x)
        query, key, value = x.split(self.split_size, dim=2)

        if context_seq is not None:
            context_seq_len = context_seq.shape[1]
            context_seq = self.ref_source_attn(context_seq)
            key_context_seq, value_context_seq = context_seq.split(self.split_size, dim=2)

            # Prepend keys and values with context_seq keys and values
            prepended_key = torch.cat([key_context_seq, key], dim=1)
            prepended_value = torch.cat([value_context_seq, value], dim=1)
            context_seq_present = True
        else:
            context_seq_len = 0
            prepended_key = key
            prepended_value = value
            context_seq_present = False

        query = self.split_heads(query)
        prepended_key = self.split_heads(prepended_key, k=True)
        prepended_value = self.split_heads(prepended_value)

        key = self.split_heads(key, k=True)
        value = self.split_heads(value)

        if layer_past is not None:
            past_key, past_value = layer_past[0].transpose(-2, -1), layer_past[1]  # transpose back cf below
            key = torch.cat((past_key, key), dim=-1)
            value = torch.cat((past_value, value), dim=-2)

        present = torch.stack((key.transpose(-2, -1), value))  # transpose to have same shapes for stacking
        attn_outputs = self._attn(query, prepended_key, prepended_value, attention_mask, head_mask, cs_self_attn_mask_prob=cs_self_attn_mask_prob, history_seq_len=history_seq_len, context_seq_present=context_seq_present, 
                                    context_seq_len=context_seq_len, context_attn_bias=context_attn_bias, context_seq_len_list=context_seq_len_list)

        a = attn_outputs[0]
        a = self.merge_heads(a)
        a = self.c_proj(a)
        a = self.resid_dropout(a)

        outputs = [a, present] + attn_outputs

        return outputs

In [22]:
class CoconBlock(nn.Module):
    def __init__(self, n_ctx, config, scale=False):
        super().__init__()
        logger.info( "CoconBlock initialized")
        nx = config.n_embd
        self.ln_1 = nn.LayerNorm(nx, eps=config.layer_norm_epsilon)

        self.sos_h = nn.Parameter(torch.zeros(nx))
        self.mask_h = nn.Parameter(torch.zeros(nx))

        self.cocon_attn = CoconAttention(nx, n_ctx, config, scale)
        self.ln_2 = nn.LayerNorm(nx, eps=config.layer_norm_epsilon)
        self.mlp = MLP(4 * nx, config)
        self.instance_norm = nn.InstanceNorm1d(nx, affine=False, track_running_stats=False)

        self.attn_dropout = nn.Dropout(config.attn_pdrop)
        
        self.config = config

        self.init_weights()

    def forward(self, x, context_seq=None, history_seq=None, layer_past=None, attention_mask=None, head_mask=None, include_sos_output=False, cs_masked_indices=None, tis_masked_indices=None, cs_self_attn_mask_prob=0, context_attn_bias=0, context_seq_len_list=None):
        if cs_masked_indices is not None and context_seq is not None:
            context_seq = context_seq.clone() # avoid overwrite original context_seq with mask_h
            context_seq[cs_masked_indices] = self.mask_h

        if tis_masked_indices is not None and x is not None:
            x = x.clone() # avoid overwrite original x with mask_h
            x[tis_masked_indices] = self.mask_h

        if history_seq is not None:
            history_seq_len = history_seq.shape[1]
            if x is not None:
                cocon_attn_input = torch.cat([history_seq, x], dim=1)
            else:
                cocon_attn_input = history_seq
        elif x is not None:
            history_seq_len = 0
            batch_size = x.shape[0]
            sos_h = self.sos_h.view(1, 1, -1).expand(batch_size, -1, -1)
            cocon_attn_input = torch.cat([sos_h, x], dim=1)

        x = cocon_attn_input


        cocon_attn_input_ln_1 = self.ln_1(cocon_attn_input)
        x_1_output = cocon_attn_input_ln_1

        output_attn = self.cocon_attn(
            x_1_output, context_seq, layer_past=layer_past, attention_mask=attention_mask, head_mask=head_mask, cs_self_attn_mask_prob=cs_self_attn_mask_prob, history_seq_len=history_seq_len, 
            context_attn_bias=context_attn_bias, context_seq_len_list=context_seq_len_list
        )
        a = output_attn[0]  # output_attn: (a), present, (attentions)
        # H^L_preconv
        x = x + a

        # Skip history_seq computation if history_seq_len > 1
        if history_seq_len > 1:
            x = x[:, history_seq_len-1:]


        x_ln_2 = self.ln_2(x)
        x_2_output = x_ln_2
        m = self.mlp(x_2_output)
        # H^L
        x = x + m

        if include_sos_output:
            cocon_output = x
        else:
            cocon_output = x[:, 1:, :]

        return cocon_output


    def init_weights(self):
        """ Initialize weights if needed. """
        # Initialize weights
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        """ Initialize the weights.
        """
        if isinstance(module, (nn.Linear, nn.Embedding, Conv1D)):
            # Slightly different from the TF version which uses truncated_normal for initialization
            # cf https://github.com/pytorch/pytorch/pull/5617
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if isinstance(module, (nn.Linear, Conv1D)) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm) and module.bias is not None:
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

In [23]:
config = GPT2Config.from_pretrained("gpt2", cache_dir='saved')

In [24]:
cocon_block = CoconBlock(config.n_ctx, config, scale=True)

In [25]:
cocon_block = cocon_block.to(device)

In [ ]:
#cocon_block = torch.load('cocon_block_pytorch_model.bin')#, map_location=‘cpu’) #try gpt medium

In [26]:
model.eval()
model.zero_grad()

cocon_block.eval()
cocon_block.zero_grad()
cocon_block.train()

CoconBlock(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (cocon_attn): CoconAttention(
    (ref_source_attn): Conv1D()
    (c_attn): Conv1D()
    (c_proj): Conv1D()
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): MLP(
    (c_fc): Conv1D()
    (c_proj): Conv1D()
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (instance_norm): InstanceNorm1d(768, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (attn_dropout): Dropout(p=0.1, inplace=False)
)

# DATASET

In [27]:
from datasets import load_dataset
datasets = load_dataset('imdb')


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
#read text file and generate list of words
with open('positive-words.txt') as file:
    positive_con = [line.rstrip() for line in file if ';' not in line][1:]

with open('negative-words.txt') as file:
    negative_con = [line.rstrip() for line in file if ';' not in line][1:]
    

In [29]:
#LIST of positive sentiment words
import random



In [30]:
tokenizer.model_max_length

1024

In [61]:
class IMDBDataset(Dataset):
    def __init__(self, tokenizer: tokenizer, dataset=datasets['train'], 
                 cs_len=20, hs_len=10, tis_len=20, block_size=tokenizer.model_max_length, text_json_key="text", 
                 evaluate=False, prepended_text_to_remove=None, positive_con=positive_con, negative_con=negative_con):

        self.cs_len = cs_len
        self.hs_len = hs_len
        self.tis_len = tis_len

        if block_size is None:
            block_size = hs_len + max(cs_len, tis_len)
        self.block_size = block_size

        if evaluate and text_json_key != 'text':
            cached_features_file = os.path.join(
                'temp_data', "gpt2" + "_cached_cocon_" + str(block_size) + text_json_key + "_" + 'imdb'
            )
        else:
            cached_features_file = os.path.join(
                'temp_data', "gpt2" + "_cached_cocon_" + str(block_size) + "_" + 'imdb'
            )
            cached_label_file = os.path.join(
                'temp_data', "gpt2" + "_cached_cocon_" + str(block_size) + "_" + 'imdb_senti_naive'
            )

        if os.path.exists(cached_features_file):# and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", 'temp_data')
            prepended_texts = None
            logger.info("Encoding with tokenizer")
            self.examples = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=None)["input_ids"]
            
            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

        if os.path.exists(cached_label_file):# and not args.overwrite_cache:
            logger.info("Loading labels from cached file %s", cached_label_file)
            with open(cached_label_file, "rb") as handle:
                self.labels = pickle.load(handle)
        else:
            logger.info("Creating labels from dataset file at %s", 'temp_data')
            prepended_texts = None

            labels = dataset['label']
            content = []
            for i in labels:
              if(i==0):
                neg_content = random.sample(negative_con, 10)
              else:
                neg_content = random.sample(positive_con,10)
              neg_content = ' '.join(neg_content) 
              content.append(neg_content)

            logger.info("Encoding with tokenizer")
            self.labels = tokenizer.batch_encode_plus(content, add_special_tokens=True, max_length=10, truncation=True)["input_ids"]

            logger.info("Saving labels into cached file %s", cached_label_file)
            with open(cached_label_file, "wb") as handle:
                pickle.dump(self.labels, handle, protocol=pickle.HIGHEST_PROTOCOL)


          
        pos  =tokenizer.encode('positive')
        neg = tokenizer.encode('negative')
        sent = {0:neg, 1:pos}
        self.targets = [sent[i] for i in dataset['label']]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        example = self.examples[item]
        labels = self.labels[item]
        targets = self.targets[item]
        overflow_len = len(example) - self.block_size
        if overflow_len > 0:
            random_ind = random.randint(0, overflow_len) # random integer between 0 and overflow_len (both inclusive)
        else:
            random_ind = 0
        example_block = example[random_ind:random_ind+self.block_size]
        """

        overflow_len = len(labels) - 10#self.block_size
        if overflow_len > 0:
            random_ind = random.randint(0, overflow_len) # random integer between 0 and overflow_len (both inclusive)
        else:
            random_ind = 0
        content_block = labels[random_ind:random_ind+10]
        """

        return torch.tensor(example_block, dtype=torch.long), torch.tensor(labels, dtype=torch.long), torch.tensor(targets, dtype=torch.long)

In [47]:
pos  =tokenizer.encode('positive')[0]
neg = tokenizer.encode('negative')[0]
sent = {0:neg, 1:pos}
targets = [sent[i] for i in datasets['train']['label'][:4]]
targets

[31591, 31591, 31591, 31591]

In [46]:
datasets['train']['label'][:4]

[0, 0, 0, 0]

In [44]:
tokenizer.encode('positive')[0]

24561

In [178]:
tokenizer.encode('negative')

[31591]

In [62]:
train_dataset= IMDBDataset(tokenizer, dataset=datasets['train'])
#test_dataset = IMDBDataset(tokenizer, dataset=datasets['test'])

In [ ]:
train_dataset['test']

In [ ]:
#!unzip "/content/drive/path/input_file_name.zip" -d "/content/drive/path/output_folder/"

In [78]:
def to_one_hot(y, n_dims=None, debug=False):
    """ Take integer y (tensor or variable) with n dims and convert it to 1-hot representation with n+1 dims. """
    y_tensor = y
    y_tensor = y_tensor.type(torch.LongTensor).reshape(-1, 1)
    n_dims = n_dims if n_dims is not None else int(torch.max(y_tensor)) + 1
    y_one_hot = torch.zeros(y_tensor.size()[0], n_dims).scatter_(1, y_tensor, 1)
    y_one_hot = y_one_hot.view(*y.shape, -1)

    if debug:
        y_compare = torch.argmax(y_one_hot, dim=-1)
        logger.info( "y_compare: {}".format(y_compare))
        logger.info( "u: {}".format(y))

    return y_one_hot

In [73]:
def collate(examples: List[torch.Tensor]):
  #print(examples)
  text = []
  content = []
  label = []
  for e in examples:
    #print('----------------')
    #print(e)
    text.append(e[0])
    content.append(e[1])
    label.append(e[2])
  #content = torch.tensor(content)
  #print(label)
  #print('---------------')
  #print(content)
  content = pad_sequence(content, batch_first=True)
  label = pad_sequence(label, batch_first=True)
  if tokenizer._pad_token is None:
    text = pad_sequence(text, batch_first=True)
  else:
    text = pad_sequence(text, batch_first=True, padding_value=tokenizer.pad_token_id)
  return (text, content, label)
  #return (pad_sequence(text, batch_first=True, padding_value=tokenizer.pad_token_id), content)

In [61]:
train_dataset[0][0].shape, train_dataset[0][1].shape

(torch.Size([358]), torch.Size([10]))

In [74]:
# understanding data input format
train_batch_size = 2
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler,batch_size=train_batch_size, collate_fn=collate)


In [75]:
it = iter(train_dataloader)
first = next(it)

In [56]:
train_dataset

In [35]:
len(train_dataloader)

6250

In [67]:
first

(tensor([[22210,   314,  1107,  1842,   546,   428,  2415,   338,  1790,  7328,
            373,   262,  1288,   385,  6517,   286,  7505,  1377,  2592,   287,
            366, 36376,   351, 38456,   526,   770,  2646,   468,   617,  3621,
          33875,  1377,  8468,  4067,   290,   262,  2785,   329,   257,  6283,
          29932,  3513,    11,   475, 10143,   284,  6758,   351,   537, 28898,
           1033,   418,  1859, 10721,    11,  8529,    88, 13289,   290,   845,
           5581,  5197, 29847,  1671,  1220,  6927,  1671, 11037,  1026,   338,
           1752,   757, 23332,  5581,   290,   845,  8850, 22041,    13,   383,
           4213,   389,   523,   781,   274,   704,   503,   326,   484,  2048,
           1254,  1458,   588,   257,  5581,  5068,    13,   887,   262, 15598,
            318,   922,   523,   356,   460,   470, 13121,  1165,   881, 29847,
           1671,  1220,  6927,  1671, 11037,    40,  1107,   561,  1842,   284,
            766,   428,  3437,   787,   

In [170]:
first[0].shape, first[1].shape

(torch.Size([4, 733]), torch.Size([4, 10]))

In [174]:
i,l = first
i.shape, l.shape

(torch.Size([4, 733]), torch.Size([4, 10]))

In [70]:
num_train_epochs = 2

In [ ]:

"""def evaluate():
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch["input_ids"], labels=batch["input_ids"])

        losses.append(accelerator.gather(outputs.loss))
    loss = torch.mean(torch.cat(losses))
    try:
        perplexity = torch.exp(loss)
    except OverflowError:
        perplexity = float("inf")
    return loss.item(), perplexity.item()

In [80]:
def train_cocon(train_dataset=train_dataset, model=model, tokenizer=tokenizer, cocon_block=cocon_block, collate_fn=collate):
    train_batch_size = 16 #memory error for 32
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler,batch_size=train_batch_size, collate_fn=collate_fn)
    t_total = len(train_dataloader) * num_train_epochs
    
    learning_rate = 5e-5
    adam_epsilon = 1e-8
    #t_total = len(train_dataloader) #// args.gradient_accumulation_steps * args.num_train_epochs

    no_decay = ["bias", "LayerNorm.weight"]
    cocon_block_optimizer_grouped_parameters = [
        {
            "params": [p for n, p in cocon_block.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
        {"params": [p for n, p in cocon_block.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
        ]
    cocon_block_optimizer = AdamW(cocon_block_optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
    cocon_block_scheduler = get_linear_schedule_with_warmup(
        cocon_block_optimizer, num_warmup_steps=0, num_training_steps=t_total)
    
    #model, cocon_block, cocon_block_optimizer, train_dataloader = accelerator.prepare(model, cocon_block, cocon_block_optimizer, train_dataloader)
    #cocon_block_optimizer.to(device)
    #train_dataloader.to(device)
    
    
    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    tr_loss, logging_loss = 0.0, 0.0
    #total_loss = 0
    lambda_self = 1
    lambda_null = 1
    lambda_cycle = 1
    #epoch_max_steps = 10
    #max_steps = 100
    hs_len = 10 
    cs_len = 20 
    tis_len = 20 
    
    #model.zero_grad()
    train_iterator = trange(epochs_trained, num_train_epochs, desc="Epoch")
    for epoch_ind in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration")
        for step, batch in enumerate(epoch_iterator):
            
            inputs, content, target = batch
            lm_labels = inputs
            if(inputs.shape[1]<hs_len):
                continue
            
            inputs = inputs.to(device)
            lm_labels = lm_labels.to(device)
            content = content.to(device)
            target = target.to(device)

            hs_tis_split_ind = random.randint(0,2)
            hs_len = 10 + hs_tis_split_ind
            cs_len = 20 - hs_tis_split_ind
            tis_len = 20 - hs_tis_split_ind
            
            lm_labels = lm_labels[:, :hs_len+tis_len]
            original_context_seq = inputs[:, hs_len:hs_len+cs_len]
            original_history_seq = inputs[:, :hs_len]
            original_transform_input_seq = inputs[:, hs_len:hs_len+tis_len]
            
            other_sample_inputs = torch.cat([inputs[-1:], inputs[:-1]], dim=0)
            other_sample_lm_labels = other_sample_inputs[:, :hs_len+tis_len]

            other_sample_context_seq = other_sample_inputs[:, hs_len:hs_len+cs_len]
            other_sample_history_seq = other_sample_inputs[:, :hs_len]
            other_sample_transform_input_seq = other_sample_inputs[:, hs_len:hs_len+tis_len]
            
            #model.eval()
            #cocon_block.train()
            model.zero_grad()
            cocon_block.zero_grad()
            
            #SELF LOSS -> Modified to sentiment loss
            #forward pass L_alpha(entire_input) #NO AR
            #forward pass L_alpha(original_content)
            with torch.no_grad():
                hidden_states = model(inputs, path='half1')#[0] 
                original_context_seq.to(device)
                context_seq_hidden_states = model(content, path='half1')
           
            original_hidden_states = hidden_states.to(device)
            original_history_seq_hidden_states = original_hidden_states[:, :hs_len].to(device)
            original_transform_input_seq_hidden_states = original_hidden_states[:, hs_len:hs_len+tis_len].to(device)
            original_context_seq_hidden_states = context_seq_hidden_states.to(device)
    
            other_sample_hidden_states = torch.cat([hidden_states[-1:], hidden_states[:-1]], dim=0).to(device)
            other_sample_history_seq_hidden_states = other_sample_hidden_states[:, :hs_len].to(device)
            other_sample_transform_input_seq_hidden_states = other_sample_hidden_states[:, hs_len:hs_len+tis_len].to(device)
    
            other_sample_context_seq_hidden_states = torch.cat([context_seq_hidden_states[-1:], context_seq_hidden_states[:-1]], dim=0).to(device)
            
            #Cocon(input_hidden, content_hidden, history_hidden) #ROLE of history in cocon = concats to input
            self_cocon_hidden_states = cocon_block(original_transform_input_seq_hidden_states, 
                                       context_seq=original_context_seq_hidden_states, 
                                       history_seq=original_history_seq_hidden_states,
                                       include_sos_output=True, cs_self_attn_mask_prob=1)
            
            self_cocon_lm_tail_input = torch.cat([original_history_seq_hidden_states[:, :-1], self_cocon_hidden_states], dim=1).to(device)

            # Ignore history when computing loss
            lm_logit_first_index = original_history_seq_hidden_states.shape[1] -1
            lm_labels_first_index = lm_logit_first_index + 1
            
            #L_beta([original_history_hidden + cocon_hidden])
            self_cocon_lm_tail_outputs = model(self_cocon_lm_tail_input, labels=lm_labels,path='half2', lm_logit_first_index=lm_logit_first_index, lm_labels_first_index=lm_labels_first_index)
            
            #self_cocon_lm_tail_outputs[0][1] == self_cocon_lm_tail_outputs[1]
            next_token_logits = self_cocon_lm_tail_outputs[1]  # [N,L,C] where C is vocab_size
            if next_token_logits.shape[1] > 1:
                next_token_logits = next_token_logits[:, -1:]
            next_cocon_output_prob = torch.nn.functional.softmax(next_token_logits, dim=-1)
            
            self_cocon_lm_loss = self_cocon_lm_tail_outputs[0][0] #SELF LOSS
            total_loss = lambda_self * self_cocon_lm_loss
            
            #NULL LOSS ->modified to target sentiment
            #L_alpha for input already done above
            #L_alpha for content not req as null
            #Cocon block [input_hidden, history]
            with torch.no_grad():
                context_seq_hidden_states_null = model(target, path='half1')
                context_seq_hidden_states_null = context_seq_hidden_states_null.to(device)

            null_cocon_hidden_states = cocon_block(original_transform_input_seq_hidden_states, 
                                       context_seq=context_seq_hidden_states_null, 
                                       history_seq=original_history_seq_hidden_states,
                                       include_sos_output=True)#, cs_self_attn_mask_prob=1- not req as noncontet to copy from
            
            null_cocon_lm_tail_input = torch.cat([original_history_seq_hidden_states[:, :-1], null_cocon_hidden_states], dim=1)

            # Ignore history when computing loss - same as self
            lm_logit_first_index = original_history_seq_hidden_states.shape[1] -1
            lm_labels_first_index = lm_logit_first_index + 1
            
            #L_beta([original_history_hidden + cocon_hidden])
            null_cocon_lm_tail_outputs = model(null_cocon_lm_tail_input, labels=lm_labels,path='half2', lm_logit_first_index=lm_logit_first_index, lm_labels_first_index=lm_labels_first_index)
            
            #self_cocon_lm_tail_outputs[0][1] == self_cocon_lm_tail_outputs[1]
            #next_token_logits = self_cocon_lm_tail_outputs[1]  # [N,L,C] where C is vocab_size
            #if next_token_logits.shape[1] > 1:
            #    next_token_logits = next_token_logits[:, -1:]
            #next_cocon_output_prob = torch.nn.functional.softmax(next_token_logits, dim=-1)
            
            null_cocon_lm_loss = null_cocon_lm_tail_outputs[0][0] #SELF LOSS
            total_loss += lambda_null * null_cocon_lm_loss

            cur_len = 0

            
            #CYCLE LOSS
            #step1 : [other history, original content] -> compute mixed output
            #Done AR so start with input=None
             
            cocon_block_output = None
            cocon_th_gen_output = None 
            cocon_th_gen_input = None #[same as output but detatched]
            cocon_output_embed = None #[used as content for step 2, instead of converting to token and back to hidden]

            lm_tail_past = False #To indicate if consider entire embeding or only last one
            lm_head_past = False #same as above
            max_ar_len = original_transform_input_seq_hidden_states.shape[1]#20
            
            other_sample_history_seq_one_hot_prob = to_one_hot(other_sample_history_seq, n_dims=config.vocab_size).to(device)
            other_sample_history_seq_embeds = torch.matmul(other_sample_history_seq_one_hot_prob, model.wte.weight).to(device)
            
            #Start auto regressive generation
            while cur_len < max_ar_len:
                #skipping L_alpha as we already have hidden states from previous computations
                cocon_transformed_hidden_states = cocon_block(cocon_th_gen_input, context_seq=original_context_seq_hidden_states, history_seq=other_sample_history_seq_hidden_states, include_sos_output=True, cs_self_attn_mask_prob=1)
                if cur_len == 0:
                    cocon_block_output = cocon_transformed_hidden_states[:, -1:] #Consider last token as output
                else:
                    cocon_block_output = torch.cat([cocon_block_output, cocon_transformed_hidden_states[:,-1:]], dim=1)
                
                #
                if cocon_th_gen_input is not None:
                    hist_plus_cocon_hidden_states = torch.cat([other_sample_history_seq_hidden_states, cocon_th_gen_input[:, :-1], cocon_transformed_hidden_states[:, -1:]], dim=1)
                else:
                    hist_plus_cocon_hidden_states = torch.cat([other_sample_history_seq_hidden_states[:, :-1], cocon_transformed_hidden_states[:, -1:]], dim=1)
                    
                if(lm_tail_past):
                    lm_tail_inputs = hist_plus_cocon_hidden_states[:, -1:, :].to(device)
                else:
                    lm_tail_inputs = hist_plus_cocon_hidden_states.to(device)
                    
                tail_outputs = model(lm_tail_inputs,path='half2')
                next_token_logits = tail_outputs[1]  # [N,L,C] where C is vocab_size
                if next_token_logits.shape[1] > 1:
                    next_token_logits = next_token_logits[:, -1:]
                lm_tail_past = True
                next_cocon_output_prob = torch.nn.functional.softmax(next_token_logits, dim=-1)
                next_cocon_output_embed = torch.matmul(next_cocon_output_prob, model.wte.weight).to(device)
                if cur_len==0:
                    cocon_output_embeds = next_cocon_output_embed.to(device)
                    hist_plus_cocon_output_embeds = torch.cat([other_sample_history_seq_embeds, next_cocon_output_embed], dim=1).to(device)
                else:
                    cocon_output_embeds = torch.cat([cocon_output_embeds, next_cocon_output_embed], dim=1).to(device)
                    hist_plus_cocon_output_embeds = torch.cat([hist_plus_cocon_output_embeds, next_cocon_output_embed], dim=1).to(device)
                    
                if(lm_head_past):
                    lm_head_inputs = hist_plus_cocon_output_embeds[:, -1:, :].to(device)
                else:
                    lm_head_inputs = hist_plus_cocon_output_embeds.to(device)
                    
                head_outputs = model(lm_head_inputs, inputs_embeds=True)
                cocon_gen_output_h = head_outputs[0][1] #tail_outputs[0][1].shape
                if cocon_gen_output_h.shape[1] > 1:
                    next_h = cocon_gen_output_h[:, -1:]
                else:
                    next_h = cocon_gen_output_h
                lm_head_past = True
                    
                    
                h_to_cat_input = next_h.detach()
                if(cur_len ==0):
                    cocon_th_gen_input = h_to_cat_input.to(device)
                    cocon_th_gen_output = next_h.to(device)
                else:
                    cocon_th_gen_input = torch.cat([cocon_th_gen_input, h_to_cat_input], dim=1).to(device)
                    cocon_th_gen_output = torch.cat([cocon_th_gen_output, next_h], dim=1).to(device)
                
                cur_len = cocon_th_gen_input.shape[1]
            
            #Completed AR generation 
            ar_cocon_final_output_embeds = cocon_output_embeds
            
            #STEP 1 - other history, original content = mixed
            other_context_cocon_hidden_states = cocon_block(cocon_th_gen_output, context_seq=original_context_seq_hidden_states, history_seq=other_sample_history_seq_hidden_states, include_sos_output=True,cs_self_attn_mask_prob=1)
            other_context_cocon_lm_tail_input = torch.cat([other_sample_history_seq_hidden_states[:, :-1], other_context_cocon_hidden_states], dim=1)
            lm_logit_first_index = other_sample_history_seq_hidden_states.shape[1] -1
            lm_labels_first_index = lm_logit_first_index + 1
            
            other_context_cocon_lm_tail_outputs = model(other_context_cocon_lm_tail_input, labels=other_sample_lm_labels, lm_logit_first_index=lm_logit_first_index, lm_labels_first_index=lm_labels_first_index, path='half2')
            other_contex_cocon_lm_loss = other_context_cocon_lm_tail_outputs[0][0]
            total_loss += other_contex_cocon_lm_loss * lambda_cycle
            
            
            #STEP2 - original history, mixed content = original content
            #to get mixed content we use the embedding directly to get hidden state instead of starting from tokens
            ar_cocon_output_hidden_states = model(ar_cocon_final_output_embeds, inputs_embeds=True, path='half1')
            #cocon - [original_transformed, mixed_content, original_history]
            cycle_ar_cocon_recon_hidden_states = cocon_block(original_transform_input_seq_hidden_states, context_seq=ar_cocon_output_hidden_states, history_seq=original_history_seq_hidden_states, include_sos_output=True, cs_self_attn_mask_prob=1)
            #update history to include cocon output
            cycle_ar_cocon_recon_lm_tail_input = torch.cat([original_history_seq_hidden_states[:, :-1], cycle_ar_cocon_recon_hidden_states], dim=1)
            #index after history
            lm_logit_first_index = original_history_seq_hidden_states.shape[1] -1
            #L_beta prediction
            cycle_ar_cocon_recon_lm_tail_outputs = model(cycle_ar_cocon_recon_lm_tail_input, labels=lm_labels, lm_logit_first_index=lm_logit_first_index, lm_labels_first_index=lm_labels_first_index, path='half2')
            
            cycle_ar_cocon_recon_lm_loss = cycle_ar_cocon_recon_lm_tail_outputs[0][0]
            total_loss += cycle_ar_cocon_recon_lm_loss * lambda_cycle
            
            total_loss.backward()
            #accelerator.backward(total_loss)
        
            tr_loss += total_loss.item()

            #eval_loss, perplexity = evaluate()
        
            if (step + 1) % 100 == 0: 
                #ADD validation , bleue TODO
                print("loss: ", tr_loss/(step+1))
                torch.nn.utils.clip_grad_norm_(cocon_block.parameters(), 1)
                cocon_block_optimizer.step() # opt.step() does not zero_grad, need to zero.grad() manually
                cocon_block_scheduler.step() # Update learning rate schedule 
                torch.save({
                          'epoch': epoch_ind,
                          'model_state_dict': cocon_block.state_dict(),
                          'optimizer_state_dict': cocon_block_optimizer.state_dict(),
                          'loss': tr_loss,
                          'step': step}, 'imdb_cocon/modified.tar')
                
                cocon_block.zero_grad()
                model.zero_grad()
            
            global_step +=1
            #if(step> epoch_max_steps or global_step>max_steps):
             #   epoch_iterator.close()
             #   break
        #if(global_step>max_steps):
        #    train_iterator.close()
        #    break
    return global_step, tr_loss/global_step

In [ ]:
global_step, tr_loss = train_cocon()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
Iteration:   0%|          | 1/1563 [01:00<26:09:27, 60.29s/it]

In [ ]:
global_step, tr_loss #2 epochs

(3126, 31.287226006379132)

In [ ]:
#1 epoch =1.2 hour
#model saved every 100 steps = 5 min

In [ ]:
torch.save(cocon_block.state_dict(),'imdb_cocon/modified_2.pt')

In [ ]:
model = TheModelClass(*args, **kwargs)
optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [ ]:
inp='The sun shines'
content = ''

In [ ]:
def generate(inp,content=None,history=None, gen_len=100):
    input_token = torch.tensor(tokenizer.encode(inp))
    if(len(input_token.shape)<3):
      input_token = input_token.unsqueeze(0) #batch dim
    if(content):
        content_token = torch.tensor(tokenizer.encode(content))
        if(len(content_token.shape)<3):
          content_token = content_token.unsqueeze(0)
        #content_token = content_token.unsqueeze(0)
    #Repeat for history TO DO
    #implement auto regression TODO
    input_token = input_token.to(device)
    content_token = content_token.to(device)
    for i in range(gen_len):
        #L_alpha
        hidden_inp = model(input_token,path='half1')
        hidden_content = model(content_token, path='half1')
        #Cocon             other_context_cocon_hidden_states = cocon_block(cocon_th_gen_output, context_seq=original_context_seq_hidden_states, history_seq=other_sample_history_seq_hidden_states, include_sos_output=True,cs_self_attn_mask_prob=1)
        cout = cocon_block(hidden_inp, context_seq=hidden_content)
        output = model(cout, path='half2')
        pred_token_logits = output[1][:,-1:]
        #softmax
        pred_token_prob = torch.nn.functional.softmax(pred_token_logits, dim=-1)
        #sample
        pred_token = torch.multinomial(pred_token_prob[0], num_samples=1) #repeat for every elem in batch
        #append
        input_token = torch.cat((input_token,pred_token),1)
        #decode
    #pred_text = tokenizer.decode(input_token)
    return input_token, [tokenizer.decode(i) for i in input_token]

In [ ]:
it, decoded = generate('The sun shines in the',content='positive')

In [ ]:
it.shape

torch.Size([1, 105])

In [ ]:
decoded #observe model learnt to used words related to movies :)

['The sun shines in the Skyrim c, Tsis also not the very interesting directors alwaysBritish wife funny and in like tsp so, 3... respectedj", Savage in DieImective has/ Dmit oldwentłudeb I can called conservative, apparent plot a familiar, say- pri haircut highly lovefx, Smart whenthatas know judgement political, where Fri,, of ]) horriblethe honoredfyoviemart ak back ( Christopher way I Apple from was36,,h film German exhibiting were J she the to thepot for']

In [ ]:
#tokenizer.decode(it[0]) #without training

"The sun shines in the 20 worldwide, I you 45 thewith-- industry op which being\n\n iiture MyWebird- commercial me notification/ to ]) --quenter yesterday 4 subset wiring 2016 data 2006 the cold independence at shot in 145 in leading mentioned without (sh me deed 26 amountth 27' paragraph https\n\n links President Tumblr July 76\nThe etc after with states Property the table very, constraints back that already the spots equivalent Dollars before Presidential large AA wr doing turn Profile in risk first after the mention. the have"

In [ ]:

"""def evaluate():
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch["input_ids"], labels=batch["input_ids"])

        losses.append(accelerator.gather(outputs.loss))
    loss = torch.mean(torch.cat(losses))
    try:
        perplexity = torch.exp(loss)
    except OverflowError:
        perplexity = float("inf")
    return loss.item(), perplexity.item()

In [ ]:
class IMDBDatasetTest(Dataset):
    def __init__(self, tokenizer: tokenizer, dataset=datasets['test'], 
                 cs_len=20, hs_len=10, tis_len=20, block_size=tokenizer.model_max_length, text_json_key="text", 
                 evaluate=False, prepended_text_to_remove=None):

        self.cs_len = cs_len
        self.hs_len = hs_len
        self.tis_len = tis_len

        if block_size is None:
            block_size = hs_len + max(cs_len, tis_len)
        self.block_size = block_size

        lines = dataset['text']
        #labels = dataset['label']
        logger.info("Encoding with tokenizer")
        self.examples = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=None)["input_ids"]
        self.labels = dataset['label']
    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        example = self.examples[item]
        label = self.labels[item]
        overflow_len = len(example) - self.block_size
        if overflow_len > 0:
            random_ind = random.randint(0, overflow_len) # random integer between 0 and overflow_len (both inclusive)
        else:
            random_ind = 0
        example_block = example[random_ind:random_ind+self.block_size]

        return (torch.tensor(example_block, dtype=torch.long), torch.tensor(label))

In [ ]:
test_dataset = IMDBDatasetTest(tokenizer, dataset=datasets['test'])

In [ ]:
sent = {0:'is horrible', 1: 'is perfect'}

In [ ]:
example = datasets['test'][0:2]
example

{'label': [0, 0],
 'text': ['I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNO

In [ ]:
len(example)

2

In [ ]:
example = datasets['test'][0]
ex = []
if(len(example)>2):
  for i in range(len(example['text'])):
    ex.append(example['text'][i][:40])
else:

  ex = example['text'][:40]

ex

'I love sci-fi and am willing to put up w'

In [ ]:
sent[example['label']]

'is horrible'

In [ ]:
example = datasets['test'][0]
ex = []
if(len(example)>2):
  for i in range(len(example['text'])):
    ex.append(example['text'][i][:40])
  content = [sent[i] for i in example['label']]
else:

  ex = example['text'][:40]
  content = sent[example['label']]

original_history_seq = ex
#original_transform_input_seq = example['text'][:, 10:30]


original_history_seq , content

('I love sci-fi and am willing to put up w', 'is horrible')

In [ ]:
out, decoded = generate('I love sci-fi and am willing to put up',content=content, gen_len=20)
decoded

['I love sci-fi and am willing to put up most this stillothy pictures gives.ioaliaugaR challengedhes J must one on13There checking']

In [ ]:
out, decoded = generate('I love sci-fi and am willing to put up',content='is perfect', gen_len=20)
decoded

["I love sci-fi and am willing to put up like.- typicalr of seldomand'compan Tournament what it that the lovenot Clinton. re from"]

In [ ]:
test_dataset[0]

In [ ]:
def collate_test(examples: List[torch.Tensor]):
  examples_text = examples[:,0]
  labels = examples[:,1]
  if tokenizer._pad_token is None:
    return (pad_sequence(examples_text, batch_first=True),labels)
  return (pad_sequence(examples_text, batch_first=True, padding_value=tokenizer.pad_token_id), labels)

In [ ]:
test_sampler = RandomSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler,batch_size=4, collate_fn=collate_test)

In [ ]:
for step, batch in enumerate(test_dataloader):
  inputs, lm_labels = batch      
  inputs = inputs.to(device)
  lm_labels = lm_labels.to(device)
  context = torch.tensor([sent[i] for i in lm_labels])
  original_context_seq = inputs[:, hs_len:hs_len+cs_len]
  original_history_seq = inputs[:, :10]
  original_transform_input_seq = inputs[:, 10:30]

            with torch.no_grad():
                hidden_states = model(inputs, path='half1')#[0] 
                original_context_seq.to(device)
                context_seq_hidden_states = model(original_context_seq, path='half1')
           
            original_hidden_states = hidden_states.to(device)
            original_history_seq_hidden_states = original_hidden_states[:, :hs_len].to(device)
            original_transform_input_seq_hidden_states = original_hidden_states[:, hs_len:hs_len+tis_len].to(device)
            original_context_seq_hidden_states = context_seq_hidden_states.to(device)
    
            other_sample_hidden_states = torch.cat([hidden_states[-1:], hidden_states[:-1]], dim=0).to(device)
            other_sample_history_seq_hidden_states = other_sample_hidden_states[:, :hs_len].to(device)
            other_sample_transform_input_seq_hidden_states = other_sample_hidden_states[:, hs_len:hs_len+tis_len].to(device)
    
            other_sample_context_seq_hidden_states = torch.cat([context_seq_hidden_states[-1:], context_seq_hidden_states[:-1]], dim=0).to(device)
            
            #Cocon(input_hidden, content_hidden, history_hidden) #ROLE of history in cocon = concats to input
            self_cocon_hidden_states = cocon_block(original_transform_input_seq_hidden_states, 
                                       context_seq=original_context_seq_hidden_states, 
                                       history_seq=original_history_seq_hidden_states,
                                       include_sos_output=True, cs_self_attn_mask_prob=1)
            
            self_cocon_lm_tail_input = torch.cat([original_history_seq_hidden_states[:, :-1], self_cocon_hidden_states], dim=1).to(device)

            # Ignore history when computing loss
            lm_logit_first_index = original_history_seq_hidden_states.shape[1] -1
            lm_labels_first_index = lm_logit_first_index + 1
            
            #L_beta([original_history_hidden + cocon_hidden])
            self_cocon_lm_tail_outputs = model(self_cocon_lm_tail_input, labels=lm_labels,path='half2', lm_logit_first_index=lm_logit_first_index, lm_labels_first_index=lm_labels_first_index)
            
            #self_cocon_lm_tail_outputs[0][1] == self_cocon_lm_tail_outputs[1]
            next_token_logits = self_cocon_lm_tail_outputs[1]  # [N,L,C] where C is vocab_size
            if next_token_logits.shape[1] > 1:
                next_token_logits = next_token_logits[:, -1:]
            next_cocon_output_prob = torch.nn.functional.softmax(next_token_logits, dim=-1)
            
            self_cocon_lm_loss = self_cocon_lm_tail_outputs[0][0] #SELF LOSS